In [8]:
%matplotlib inline

import random
import classifier
reload(classifier)

import pandas as pd
import numpy
from IPython.display import display
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from classifier import encode_onehot
from sklearn.decomposition import PCA


def prepare_data(df):
    df4 = df
    df1 = pd.DataFrame(df.loc[:,[u'0',u'5',u'7',u'8',u'9',u'14',u'16',u'17',u'56',u'57']])
    #0 , 26 8, 5 , 14, 23 , 58 -> boss
    #17, 56, 16,9,, 25, 57 -> yes
    #20, 18, 7 -> no
    #df2 = pd.DataFrame(df.loc[:,[u'18',u'20',u'23',u'25',u'26']])
    df2 = pd.DataFrame(df.loc[:,[u'18',u'20',u'23',u'25',u'26',u'58']])
    df3 = pd.DataFrame(df.drop([u'0',u'5',u'7',u'8',u'9',u'14',u'16',u'17',
                          u'18',u'20',u'23',u'25',u'26',u'56',u'57',u'58'],axis=1))
    
    return df1, df2, df3, df4

def encode(dataset):
    drop_cols = []
    for i in dataset.columns:
        if isinstance(dataset[i].values[0], basestring):
            drop_cols = drop_cols + [i]
    
    return encode_onehot(dataset, drop_cols)

def encode_with_test(train, test):
    agg_data = pd.concat([train, test],axis=0,ignore_index=True)
    agg_data= encode(agg_data)
    
    return agg_data._slice(slice(0,train.shape[0]),axis=0), agg_data._slice(slice(train.shape[0],agg_data.shape[0]),axis=0)

train = pd.read_csv('data.csv')
label = train['label']
train = train.drop('label',1)
print len(train.columns)
X_quiz = pd.read_csv('quiz.csv')

#split the dataset
X_train, X_test, y_train, y_test = train_test_split(train, label)
train_1, train_2, train_3, train_4 = prepare_data(X_train)
test_1, test_2, test_3, test_4 = prepare_data(X_test)
quiz_1, quiz_2, quiz_3, quiz_4 = prepare_data(X_quiz)
                             
print train_4.shape
    
#agg_data = pd.concat([train, X_quiz],axis=0,ignore_index=True)


#display(agg_data.shape[0])
#train = agg_data._slice(slice(0,train.shape[0]),0)
#X_quiz = agg_data._slice(slice(train.shape[0],agg_data.shape[0]),0)
#pca = PCA(n_components=100)
#train = pca.fit(train).transform(train)
#display(train.shape)
#display(X_quiz.shape)

print len(train_1.columns) + len(train_2.columns) + len(train_3.columns)
print train.shape, train_1.shape, train_2.shape, train_3.shape

52
(95127, 52)
52
(126837, 52) (95127, 10) (95127, 6) (95127, 36)


In [10]:
import classifier
reload(classifier)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import xgboost as xgb

train_1 = encode(X_train)
X_train1, X_test1, y_train1, y_test1 = train_test_split(train_1, y_train)


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from classifier import blend_clf

train_1 = encode(X_train)
X_train1, X_test1, y_train1, y_test1 = train_test_split(train_1, y_train)

#gbc = GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)
clfs = [xgb.XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=7),
        RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini', max_features='sqrt'),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy', max_features='sqrt')]


predictions = blend_clf(clfs, X_train1.as_matrix(),X_test1.as_matrix(),y_train1.as_matrix())
print classification_report(y_test1, predictions,digits=4)



Creating train and test sets for blending.
0 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=

In [22]:
import classifier
reload(classifier)

from classifier import train_SGD_SVM
from sklearn.metrics import classification_report

svm_clf = train_SGD_SVM(X_train3, y_train3)
predictions = svm_clf.predict(X_test3)
print classification_report(y_test3, predictions)

['clf__average', 'clf__l1_ratio', 'clf__penalty', 'clf__warm_start', 'clf__random_state', 'clf', 'clf__n_iter', 'clf__learning_rate', 'clf__verbose', 'steps', 'clf__loss', 'clf__class_weight', 'clf__fit_intercept', 'clf__power_t', 'clf__n_jobs', 'clf__eta0', 'clf__alpha', 'clf__shuffle', 'clf__epsilon']
Fitting 3 folds for each of 2 candidates, totalling 6 fits
-- Epoch 1
Norm: 23.02, NNZs: 33, Bias: 0.304084, T: 499, Avg. loss: 533.381897
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 11.81, NNZs: 33, Bias: -3.719816, T: 998, Avg. loss: 377.859071
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 12.63, NNZs: 33, Bias: -2.187557, T: 1497, Avg. loss: 299.342091
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 7.12, NNZs: 33, Bias: -0.819241, T: 1996, Avg. loss: 250.458674
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 12.35, NNZs: 33, Bias: -0.846817, T: 2495, Avg. loss: 216.775159
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 26.88, NNZs: 33, Bias: 0.846839, T: 50

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished


In [29]:
import classifier
reload(classifier)

from classifier import train_ada_boost
from sklearn.metrics import classification_report

dt_clf = train_ada_boost(X_train3,y_train3)
predictions = dt_clf.predict(X_test3)
print classification_report(y_test3, predictions)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished


Best score: 0.681
Best parameters set:
	clf__n_estimators: 100
             precision    recall  f1-score   support

         -1       0.69      0.80      0.74       133
          1       0.72      0.60      0.65       117

avg / total       0.71      0.70      0.70       250



In [15]:
import classifier
reload(classifier)

from sklearn.ensemble import ExtraTreesClassifier
from classifier import train_xtree_classifer, train_rf, train_SGD_SVM, train_friedman_xgbm
from sklearn.metrics import classification_report

# XTree is okay in set 3

train_3 = encode(X_train)
X_train3, X_test3, y_train3, y_test3 = train_test_split(train_3, y_train)

xtree = ExtraTreesClassifier(n_jobs=-1,n_estimators=200,max_depth=None, max_features='sqrt',random_state=0)
xtree.fit(X_train3, y_train3)
predictions = xtree.predict(X_test3)
print classification_report(y_test3, predictions, digits=4)

             precision    recall  f1-score   support

         -1     0.9254    0.9680    0.9462     13349
          1     0.9565    0.9001    0.9275     10433

avg / total     0.9390    0.9382    0.9380     23782

